In [ ]:
from qdrant_client import QdrantClient


# Initialize the client
client = QdrantClient(":memory:")


In [ ]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('styles.csv', on_bad_lines='skip')

# Extract the 'productDisplayName' column into a list
product_display_names = df['productDisplayName'].astype(str).tolist()


In [ ]:
# Function to split list into batches of specified size
def batch_list(lst, batch_size):
    for i in range(0, len(lst), batch_size):
        yield lst[i:i + batch_size]

# Define batch size
batch_size = 1000

# Process the batches
for idx, batch in enumerate(batch_list(product_display_names, batch_size)):
    start_id = idx * batch_size + 1
    end_id = start_id + len(batch) - 1
    doc_ids = list(range(start_id, end_id + 1))
    client.add(
        collection_name="product_list",
        documents=batch,
        ids=doc_ids,
    )

In [ ]:
groq_model = dspy.GROQ(model='llama3-8b-8192', api_key ="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")

In [ ]:
from dspy.retrieve.qdrant_rm import QdrantRM
import dspy
from groq import Groq


qdrant_retriever_model = QdrantRM("product_list", client, k=3)


ollama_model = dspy.OllamaLocal(model="llama3",model_type='text',
                                max_tokens=350,
                                temperature=0.1,
                                top_p=0.8, frequency_penalty=1.17, top_k=40)

dspy.settings.configure(lm= ollama_model, rm=qdrant_retriever_model)

In [ ]:
class Items(dspy.Signature):
    """
    Extract clothing items and accessories given user preference
    """
    prompt = dspy.InputField()
    terms = dspy.OutputField(format=list)


class FindItems(dspy.Module):

    def __init__(self):
        super().__init__()
        self.item_extractor = dspy.Predict(Items)

    def forward(self, question):
        instruction = (
            "Return a list of clothing items and accesores to be worn by a user, given their preferences"
            f"Preferences: {question}"
        )
        prediction = self.item_extractor(
            prompt=instruction
        )
        return prediction.terms

In [53]:
module = FindItems()

In [54]:
response = module("Tripy party with clowns and lot if music in New York")

In [55]:
print(response)

Here is the extracted list of clothing items and accessories:

**Clothing Items**

1. Sequined dress (to match the flashy atmosphere)
2. Black jeans (for a stylish contrast to the colorful setting)

**Accessories**

1. Glittery earrings (to add some sparkle to their outfit)
2. Clown-themed necklace or pin (as a nod to the party theme)
3. Statement belt with flashing lights or sequins
4. High-heeled shoes in bold colors like red, blue, or green

These items are designed to help the user stand out at the trippy party while still looking stylish and put-together!


In [138]:
trainset_dict= {
    "desert_wear": "* Lightweight, breathable long-sleeve shirt\n* Wide-brimmed hat\n* Sunglasses with UV protection\n* Cargo pants\n* Lightweight scarf or shemagh\n* Hiking boots with good ventilation\n* Moisture-wicking socks\n* Sunscreen\n* Hydration pack or water bottle\n* Lightweight jacket for cooler evenings",
    "caribbean_party": "* Flowy sundress with tropical print\n* Beach hat with colorful feathers\n* Statement necklace with seashells and beads\n* Flip flops in bright colors\n* Sunglasses with brightly colored frames\n* Lightweight, short-sleeve shirt with floral patterns\n* Comfortable shorts\n* Colorful bangles or bracelets\n* Straw tote bag\n* Lightweight scarf or wrap",
    "award_function": "* Elegant evening gown or tuxedo\n* Statement jewelry (earrings, necklace, bracelet)\n* Dress shoes or heels\n* Clutch or small evening bag\n* Cufflinks and tie (for tuxedo)\n* Evening makeup and hairstyle\n* Silk scarf or shawl\n* Formal watch\n* Perfume or cologne\n* Pocket square (for tuxedo)"
}

In [139]:
trainset = [dspy.Example(question=key, answer=value) for key, value in trainset_dict.items()]

In [141]:
from dspy import teleprompt
teleprompter = teleprompt.LabeledFewShot()
optimized_Find_Items = teleprompter.compile(student=FindItems(), trainset=trainset)

In [164]:
response = optimized_Find_Items("party wear")

In [165]:
print(response)


Here is the extracted list of clothing items and accessories:

**Party Wear**

* Sequined dress
* High heels (black or red)
* Statement necklace with rhinestones
* Earrings with dangling crystals
* Sparkly clutch purse (silver or gold)
* Cocktail ring with gemstones


Bad pipe message: %s [b"\xc0\x0f\xe1\xa8\xa3\xfb\x06\x18L\x8a\x00\xafGf\xca[d\xee\x00\x00|\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R\xc0$\xc0(\x00k\x00j\xc0#\xc0'\x00g\x00@\xc0\n\xc0"]
Bad pipe message: %s [b'9\x008\xc0\t\xc0\x13\x003\x002\x00\x9d\xc0\xa1\xc0\x9d\xc0Q']
Bad pipe message: %s [b"v\xf08\xe8\x14\xddTb\x869\xb9\x00\xefl_Z\x9a\xc0\x00\x00\xa6\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R\xc0$\xc0(\x00k\x00j\xc0s\xc0w\x00\xc4\x00\xc3\xc0#\xc0'\x00g\x00@\xc0r\xc0v\x00\xbe\x00\xbd\xc0\n\xc0\x14\x009\x008\x00\x88\x00\x87\xc0\t\xc0\x13\x003\x002\x00\x9a\x00\x99\x00E\x00D\xc0\x07\xc0\x11\xc0\x08\xc0\x12\x00\x16\x00\x13\x00\x9d\xc0\xa1\xc0\x9d\xc0Q\x00\x9c\xc0\xa0\xc0\x9c\xc0P\x00=\x00\xc0\x00<\x00\xba\x005\x00\x

In [130]:
parse_clothing_items(module)

['Snake-print scarf',
 'Black leather jacket with silver studs',
 'High-waisted black jeans with ripped knees',
 'White tank top with built-in bralette',
 'Ankle boots with buckles shaped like snake heads']

In [ ]:
import re

def parse_clothing_list(text):
    # Regular expression to match items after numbers
    pattern = r'\d+\.\s(.+)'

    # Find all matches in the input text
    matches = re.findall(pattern, text)

    return matches



In [144]:
def parse_clothing_items(text):
    # Split the text into lines
    lines = text.split('\n')

    # Initialize an empty list to hold the items
    items = []

    # Loop through each line
    for line in lines:
        # Check if the line starts with '*'
        if line.strip().startswith('* '):
            # Remove the '*' and any leading/trailing whitespace, then add to the list
            item = line.strip()[2:].strip()
            items.append(item)

    return items


In [155]:
class Style(dspy.Signature):
    """
    Generate styling advise, given these products
    """
    products = dspy.InputField()
    answer = dspy.OutputField()

In [167]:
class Styling_Advise(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_response = dspy.Predict(Style)
        self.retrieve = dspy.Retrieve(k=3)

    def forward(self, question):
        item_list= optimized_Find_Items(question)
        product_list = []
        for item in item_list:
            product_list += self.retrieve(item).passages

        response = self.generate_response(products= ', '.join(product_list))
        return response.answer

In [168]:
module = Styling_Advise()

In [169]:
response = module("Corporate Award Ceremony")


In [170]:
print(response)

Based on the products you've listed, here are some styling advice:

**Casual Outfit**

* Pair Do U Speak Green Boys Olive Trousers with a matching olive green t-shirt (Do u speak green Men Olive T-shirt) and sneakers for a relaxed look.
* Add a pop of color with Red Rose Pink Camisole or any other camisole from the collection.

**Smart Casual**

* Combine Do U Speak Green Men Blue T-shirt with Vans Men Rowley shoes and dark wash jeans for a stylish, everyday outfit.
* Accessorize with Colour Me Men Silver Deo to add some shine to your look.

**Formal Occasion**

* Go classic with Nike Men's Quoted Black T-shirt paired with formal trousers (Do u speak green Boys Olive Trousers) and dress shoes.
* Add a touch of sophistication with Rasasi Men Numero Due Deo or Colour Me Women Purple Deo for a stylish fragrance.

**Layering**

* Layer Do U Speak Green Men Olive Sweatshirt over a plain white t-shirt (Tantra T-shirt) for a cozy, casual look.
* Use Lakme Insta-Liner to add some definition to

In [171]:
ollama_model.inspect_history(n=3)




List of extracted clothing items and accessories

---

Follow the following format.

Prompt: ${prompt}
Terms: ['$', '{', 't', 'e', 'r', 'm', 's', '}']

---

Prompt: Return a list of clothing items and accesores to be worn by a user, given their preferencesPreferences: party wear
Terms: [] Here is the extracted list of clothing items and accessories:

**Party Wear**

* Sequined dress
* High heels (black or red)
* Statement necklace with rhinestones
* Earrings with dangling crystals
* Sparkly clutch purse (silver or gold)
* Cocktail ring with gemstones





List of extracted clothing items and accessories

---

Follow the following format.

Prompt: ${prompt}
Terms: ['$', '{', 't', 'e', 'r', 'm', 's', '}']

---

Prompt: Return a list of clothing items and accesores to be worn by a user, given their preferencesPreferences: Corporate Award Ceremony
Terms: [] Here is the extracted list of clothing items and accessories:

**Corporate Award Ceremony**

* Suit (black or navy blue)
* Dress sh

"\n\n\nList of extracted clothing items and accessories\n\n---\n\nFollow the following format.\n\nPrompt: ${prompt}\nTerms: ['$', '{', 't', 'e', 'r', 'm', 's', '}']\n\n---\n\nPrompt: Return a list of clothing items and accesores to be worn by a user, given their preferencesPreferences: party wear\nTerms: []\x1b Here is the extracted list of clothing items and accessories:\n\n**Party Wear**\n\n* Sequined dress\n* High heels (black or red)\n* Statement necklace with rhinestones\n* Earrings with dangling crystals\n* Sparkly clutch purse (silver or gold)\n* Cocktail ring with gemstones\x1b\n\n\n\n\n\nList of extracted clothing items and accessories\n\n---\n\nFollow the following format.\n\nPrompt: ${prompt}\nTerms: ['$', '{', 't', 'e', 'r', 'm', 's', '}']\n\n---\n\nPrompt: Return a list of clothing items and accesores to be worn by a user, given their preferencesPreferences: Corporate Award Ceremony\nTerms: []\x1b Here is the extracted list of clothing items and accessories:\n\n**Corporate